In [66]:
import pandas as pd
import random
import numpy as np
import ast
from math import radians, cos, sin, asin, sqrt  
import time 
from keras.models import Sequential  
from keras.layers import Dense

In [67]:
def caltime(row):
    
    # calculate the time cost
    polylen = len(ast.literal_eval(row["POLYLINE"]))
    timecost = (polylen-1)*15
    return timecost

def snapshot(row):
    # get the polyline numbers
    snap = len(ast.literal_eval(row["POLYLINE"]))
    return snap

def initial_long(row):
    # get the initial longitude
    loc = ast.literal_eval(row["POLYLINE"])[0]
    longitude = loc[0]
    return longitude

def initial_lat(row):
    # get the initial latitude
    loc = ast.literal_eval(row["POLYLINE"])[0]
    latitude = loc[1]
    return latitude

def haversine(row): 
    """ 
    Calculate the great circle distance between two points  
    on the earth (specified in decimal degrees) 
    """   
    lon1 = row['initial_lon']
    lat1 = row['initial_lat']
    lon2 = ast.literal_eval(row["POLYLINE"])[-1][0]
    lat2 = ast.literal_eval(row["POLYLINE"])[-1][1]
    
    # change Decimal to radian   
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])  
  
    # haversine
    dlon = lon2 - lon1   
    dlat = lat2 - lat1   
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2  
    c = 2 * asin(sqrt(a))
    
    
    # The average radius of the earth, in kilometers    
    r = 6371
    diff = c * r * 1000  
    
    return diff/1000 

def get_date(row):
    tt = time.gmtime(row["TIMESTAMP"])
    time_data = [tt.tm_mon,tt.tm_mday,tt.tm_hour]
    
    return pd.Series(np.array(time_data, dtype=float))

In [68]:
filename = "D:/Program/dataset/Taxi Trip Time Prediction/train.csv"

#number of records in file (excludes header)
df_rows = sum(1 for line in open(filename)) - 1 

# Sample size - in this case ~10%
size = int(df_rows / 500)

#the 0-indexed header will not be included in the skip list
skip_idx = sorted(random.sample(range(1, df_rows), df_rows - size))

df = pd.read_csv(filename,skiprows=skip_idx)
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372651896620000496,A,59134.0,NaN,20000496,1372651896,A,False,"[[-8.628003,41.157972],[-8.62803,41.157999],[-..."
1,1372659665620000367,B,NaN,51.0,20000367,1372659665,A,False,"[[-8.621019,41.161059],[-8.621379,41.160465],[..."
2,1372655604620000007,A,59513.0,NaN,20000007,1372655604,A,False,"[[-8.637408,41.185521],[-8.637381,41.185602],[..."
3,1372673297620000607,A,5591.0,NaN,20000607,1372673297,A,False,"[[-8.660358,41.172534],[-8.660421,41.172669],[..."
4,1372683478620000640,A,2002.0,NaN,20000640,1372683478,A,False,"[[-8.585667,41.163147],[-8.586009,41.163246],[..."


In [69]:
df.shape

(3421, 9)

In [70]:
df = df.drop(["CALL_TYPE","ORIGIN_CALL","ORIGIN_STAND","TRIP_ID","DAY_TYPE","MISSING_DATA"],axis=1)
df.head()

,TAXI_ID,TIMESTAMP,POLYLINE
0,20000496,1372651896,"[[-8.628003,41.157972],[-8.62803,41.157999],[-..."
1,20000367,1372659665,"[[-8.621019,41.161059],[-8.621379,41.160465],[..."
2,20000007,1372655604,"[[-8.637408,41.185521],[-8.637381,41.185602],[..."
3,20000607,1372673297,"[[-8.660358,41.172534],[-8.660421,41.172669],[..."
4,20000640,1372683478,"[[-8.585667,41.163147],[-8.586009,41.163246],[..."


In [71]:
# normalize the taxi id
df['TAXI_ID'] -= np.min(df['TAXI_ID']) 

df["timecost"] = df.apply(caltime,axis=1)
df["snapshots"] = df.apply(snapshot,axis=1)

df.head()

,TAXI_ID,TIMESTAMP,POLYLINE,timecost,snapshots
0,495,1372651896,"[[-8.628003,41.157972],[-8.62803,41.157999],[-...",1110,75
1,366,1372659665,"[[-8.621019,41.161059],[-8.621379,41.160465],[...",465,32
2,6,1372655604,"[[-8.637408,41.185521],[-8.637381,41.185602],[...",585,40
3,606,1372673297,"[[-8.660358,41.172534],[-8.660421,41.172669],[...",1125,76
4,639,1372683478,"[[-8.585667,41.163147],[-8.586009,41.163246],[...",4875,326


In [72]:
# drop the empty polyline
empty_poly = df.loc[df['POLYLINE'] == '[]'].index.tolist()
df = df.drop(empty_poly)

# drop the snapshot less than the 4
snap_short = df.loc[df['snapshots']<4].index.tolist()
df = df.drop(snap_short)
df.head()

,TAXI_ID,TIMESTAMP,POLYLINE,timecost,snapshots
0,495,1372651896,"[[-8.628003,41.157972],[-8.62803,41.157999],[-...",1110,75
1,366,1372659665,"[[-8.621019,41.161059],[-8.621379,41.160465],[...",465,32
2,6,1372655604,"[[-8.637408,41.185521],[-8.637381,41.185602],[...",585,40
3,606,1372673297,"[[-8.660358,41.172534],[-8.660421,41.172669],[...",1125,76
4,639,1372683478,"[[-8.585667,41.163147],[-8.586009,41.163246],[...",4875,326


In [73]:
df['initial_lon'] = df.apply(initial_long,axis=1)
df['initial_lat'] = df.apply(initial_lat,axis=1)
df['diff'] = df.apply(haversine,axis=1)

ds = df.apply(get_date,axis=1)
ds.columns = ["mon","day","hour"]
df = df.join(ds)

df.head()

,TAXI_ID,TIMESTAMP,POLYLINE,timecost,snapshots,initial_lon,initial_lat,diff,mon,day,hour
0,495,1372651896,"[[-8.628003,41.157972],[-8.62803,41.157999],[-...",1110,75,-8.628003,41.157972,9.522285,7.0,1.0,4.0
1,366,1372659665,"[[-8.621019,41.161059],[-8.621379,41.160465],[...",465,32,-8.621019,41.161059,3.260045,7.0,1.0,6.0
2,6,1372655604,"[[-8.637408,41.185521],[-8.637381,41.185602],[...",585,40,-8.637408,41.185521,4.154573,7.0,1.0,5.0
3,606,1372673297,"[[-8.660358,41.172534],[-8.660421,41.172669],[...",1125,76,-8.660358,41.172534,5.222088,7.0,1.0,10.0
4,639,1372683478,"[[-8.585667,41.163147],[-8.586009,41.163246],[...",4875,326,-8.585667,41.163147,0.135672,7.0,1.0,12.0


In [74]:
# let the y_label to the last column
cols = df.columns.tolist()
y_label = cols[3]
del cols[3]
cols.append(y_label)
df = df[cols]

df = df.drop(["TIMESTAMP"],axis=1)
df = df.drop(["POLYLINE"],axis=1)
df = df.drop(["TAXI_ID"],axis=1)
df.head()

,snapshots,initial_lon,initial_lat,diff,mon,day,hour,timecost
0,75,-8.628003,41.157972,9.522285,7.0,1.0,4.0,1110
1,32,-8.621019,41.161059,3.260045,7.0,1.0,6.0,465
2,40,-8.637408,41.185521,4.154573,7.0,1.0,5.0,585
3,76,-8.660358,41.172534,5.222088,7.0,1.0,10.0,1125
4,326,-8.585667,41.163147,0.135672,7.0,1.0,12.0,4875


In [75]:
X_train,Y_train = df[df.columns[:-1]],df[df.columns[-1]]
X_train = X_train.drop(["mon","day"],axis=1)

X_train.head()

,snapshots,initial_lon,initial_lat,diff,hour
0,75,-8.628003,41.157972,9.522285,4.0
1,32,-8.621019,41.161059,3.260045,6.0
2,40,-8.637408,41.185521,4.154573,5.0
3,76,-8.660358,41.172534,5.222088,10.0
4,326,-8.585667,41.163147,0.135672,12.0


In [76]:
model = Sequential() 
model.add(Dense(units=30,input_shape=(5,),activation="relu",name="hidden_1"))
model.add(Dense(units=30,activation="relu",name="hidden_2"))
model.add(Dense(units=30,activation="relu",name="hidden_3"))
model.add(Dense(units=30,activation="relu",name="hidden_4"))
model.add(Dense(units=1,activation="relu",name="output"))
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_1 (Dense)             (None, 30)                180       
_________________________________________________________________
hidden_2 (Dense)             (None, 30)                930       
_________________________________________________________________
hidden_3 (Dense)             (None, 30)                930       
_________________________________________________________________
hidden_4 (Dense)             (None, 30)                930       
_________________________________________________________________
output (Dense)               (None, 1)                 31        
Total params: 3,001
Trainable params: 3,001
Non-trainable params: 0
_________________________________________________________________


In [77]:
model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
train_history = model.fit(X_train.values, Y_train.values, validation_split=0.2, epochs=2000, batch_size=20, verbose=1) 

Train on 2656 samples, validate on 664 samples
Epoch 1/2000
2656/2656 [==============================] - 1s - loss: 481285.6973 - acc: 0.0000e+00 - val_loss: 80546.5112 - val_acc: 0.0015
Epoch 2/2000
2656/2656 [==============================] - 0s - loss: 33462.4181 - acc: 0.0053 - val_loss: 3681.6683 - val_acc: 0.0105
Epoch 3/2000
2656/2656 [==============================] - 0s - loss: 1327.8334 - acc: 0.0184 - val_loss: 674.5454 - val_acc: 0.0136
Epoch 4/2000
2656/2656 [==============================] - 0s - loss: 556.9315 - acc: 0.0282 - val_loss: 320.7332 - val_acc: 0.0196
Epoch 5/2000
2656/2656 [==============================] - 0s - loss: 299.3651 - acc: 0.0358 - val_loss: 153.2943 - val_acc: 0.0437
Epoch 6/2000
2656/2656 [==============================] - 0s - loss: 135.5153 - acc: 0.0648 - val_loss: 78.3215 - val_acc: 0.0904
Epoch 7/2000
2656/2656 [==============================] - 0s - loss: 75.6662 - acc: 0.0794 - val_loss: 41.9018 - val_acc: 0.1295
Epoch 8/2000
2656/2656 [==

2656/2656 [==============================] - 0s - loss: 0.2201 - acc: 0.8517 - val_loss: 0.2567 - val_acc: 0.7741
Epoch 65/2000
2656/2656 [==============================] - 0s - loss: 0.1854 - acc: 0.8754 - val_loss: 0.2005 - val_acc: 0.8675
Epoch 66/2000
2656/2656 [==============================] - 0s - loss: 0.4983 - acc: 0.6547 - val_loss: 0.1809 - val_acc: 0.8645
Epoch 67/2000
2656/2656 [==============================] - 0s - loss: 0.2245 - acc: 0.8419 - val_loss: 0.3900 - val_acc: 0.3810
Epoch 68/2000
2656/2656 [==============================] - 0s - loss: 0.2396 - acc: 0.7914 - val_loss: 0.0914 - val_acc: 0.9081
Epoch 69/2000
2656/2656 [==============================] - 1s - loss: 0.3178 - acc: 0.7489 - val_loss: 0.2597 - val_acc: 0.7605
Epoch 70/2000
2656/2656 [==============================] - 0s - loss: 0.1880 - acc: 0.8385 - val_loss: 0.1098 - val_acc: 0.9127
Epoch 71/2000
2656/2656 [==============================] - 0s - loss: 0.1532 - acc: 0.8874 - val_loss: 0.1610 - val_ac

2656/2656 [==============================] - 0s - loss: 16.3034 - acc: 0.3806 - val_loss: 1.1233 - val_acc: 0.0331
Epoch 128/2000
2656/2656 [==============================] - 0s - loss: 3.0033 - acc: 0.2560 - val_loss: 2.1652 - val_acc: 0.0407
Epoch 129/2000
2656/2656 [==============================] - 0s - loss: 3.2162 - acc: 0.4277 - val_loss: 1.3959 - val_acc: 0.1054
Epoch 130/2000
2656/2656 [==============================] - 1s - loss: 0.4970 - acc: 0.6973 - val_loss: 0.1948 - val_acc: 0.8524
Epoch 131/2000
2656/2656 [==============================] - 0s - loss: 14.9104 - acc: 0.7082 - val_loss: 563.5479 - val_acc: 0.0392
Epoch 132/2000
2656/2656 [==============================] - 0s - loss: 95.8129 - acc: 0.0806 - val_loss: 4.3053 - val_acc: 0.2139
Epoch 133/2000
2656/2656 [==============================] - 0s - loss: 0.3911 - acc: 0.7210 - val_loss: 0.1159 - val_acc: 0.9202
Epoch 134/2000
2656/2656 [==============================] - 0s - loss: 0.1114 - acc: 0.9390 - val_loss: 0.0

2656/2656 [==============================] - 0s - loss: 1.0396 - acc: 0.6220 - val_loss: 3.9317 - val_acc: 0.1039
Epoch 191/2000
2656/2656 [==============================] - 0s - loss: 0.5345 - acc: 0.6645 - val_loss: 0.1673 - val_acc: 0.9021
Epoch 192/2000
2656/2656 [==============================] - 0s - loss: 0.1124 - acc: 0.9251 - val_loss: 0.0650 - val_acc: 0.9352
Epoch 193/2000
2656/2656 [==============================] - 0s - loss: 0.1104 - acc: 0.9115 - val_loss: 0.2050 - val_acc: 0.8358
Epoch 194/2000
2656/2656 [==============================] - 0s - loss: 0.0830 - acc: 0.9435 - val_loss: 0.0509 - val_acc: 0.9503
Epoch 195/2000
2656/2656 [==============================] - 0s - loss: 0.2126 - acc: 0.7843 - val_loss: 0.0946 - val_acc: 0.9488
Epoch 196/2000
2656/2656 [==============================] - 0s - loss: 0.0867 - acc: 0.9334 - val_loss: 0.0788 - val_acc: 0.9262
Epoch 197/2000
2656/2656 [==============================] - 0s - loss: 1.3027 - acc: 0.6450 - val_loss: 6.6003 -

2656/2656 [==============================] - 0s - loss: 0.2184 - acc: 0.7816 - val_loss: 0.0951 - val_acc: 0.9413
Epoch 254/2000
2656/2656 [==============================] - 0s - loss: 19.8127 - acc: 0.1803 - val_loss: 2.3464 - val_acc: 0.0979
Epoch 255/2000
2656/2656 [==============================] - 0s - loss: 1.4333 - acc: 0.4492 - val_loss: 0.0860 - val_acc: 0.9277
Epoch 256/2000
2656/2656 [==============================] - 0s - loss: 8.4704 - acc: 0.6702 - val_loss: 37.9656 - val_acc: 0.1340
Epoch 257/2000
2656/2656 [==============================] - 0s - loss: 2.2072 - acc: 0.3931 - val_loss: 11.6606 - val_acc: 0.0045
Epoch 258/2000
2656/2656 [==============================] - 0s - loss: 24.7702 - acc: 0.2346 - val_loss: 13.6266 - val_acc: 0.0151
Epoch 259/2000
2656/2656 [==============================] - 0s - loss: 3.9928 - acc: 0.3671 - val_loss: 2.5635 - val_acc: 0.3358
Epoch 260/2000
2656/2656 [==============================] - 0s - loss: 16.3025 - acc: 0.2327 - val_loss: 7.

...

2656/2656 [==============================] - 0s - loss: 90.1317 - acc: 0.0986 - val_loss: 411.9131 - val_acc: 0.0151
Epoch 1696/2000
2656/2656 [==============================] - 0s - loss: 43.8405 - acc: 0.1261 - val_loss: 3.5451 - val_acc: 0.1220
Epoch 1697/2000
2656/2656 [==============================] - 0s - loss: 0.2788 - acc: 0.8592 - val_loss: 0.0121 - val_acc: 0.9925
Epoch 1698/2000
2656/2656 [==============================] - 0s - loss: 0.0079 - acc: 0.9947 - val_loss: 0.0042 - val_acc: 0.9970
Epoch 1699/2000
2656/2656 [==============================] - 0s - loss: 0.0065 - acc: 0.9951 - val_loss: 0.0036 - val_acc: 0.9970
Epoch 1700/2000
2656/2656 [==============================] - 0s - loss: 0.0057 - acc: 0.9940 - val_loss: 0.0032 - val_acc: 0.9970
Epoch 1701/2000
2656/2656 [==============================] - 0s - loss: 0.0169 - acc: 0.9925 - val_loss: 0.1187 - val_acc: 0.9428
Epoch 1702/2000
2656/2656 [==============================] - 0s - loss: 0.0183 - acc: 0.9925 - val_los

2656/2656 [==============================] - 0s - loss: 0.2472 - acc: 0.7726 - val_loss: 0.4073 - val_acc: 0.4322
Epoch 1759/2000
2656/2656 [==============================] - 0s - loss: 36.5828 - acc: 0.2233 - val_loss: 0.9623 - val_acc: 0.6461
Epoch 1760/2000
2656/2656 [==============================] - 0s - loss: 0.4431 - acc: 0.6849 - val_loss: 0.0179 - val_acc: 0.9834
Epoch 1761/2000
2656/2656 [==============================] - 0s - loss: 0.0306 - acc: 0.9849 - val_loss: 0.0133 - val_acc: 0.9864
Epoch 1762/2000
2656/2656 [==============================] - 0s - loss: 0.0067 - acc: 0.9959 - val_loss: 0.0230 - val_acc: 0.9895
Epoch 1763/2000
2656/2656 [==============================] - 0s - loss: 0.0075 - acc: 0.9959 - val_loss: 0.0138 - val_acc: 0.9880
Epoch 1764/2000
2656/2656 [==============================] - 0s - loss: 0.0086 - acc: 0.9959 - val_loss: 0.0456 - val_acc: 0.9819
Epoch 1765/2000
2656/2656 [==============================] - 0s - loss: 0.1434 - acc: 0.8509 - val_loss: 

2656/2656 [==============================] - 0s - loss: 0.2476 - acc: 0.8053 - val_loss: 0.8432 - val_acc: 0.3253
Epoch 1822/2000
2656/2656 [==============================] - 0s - loss: 13.3107 - acc: 0.1977 - val_loss: 2.4609 - val_acc: 0.3991
Epoch 1823/2000
2656/2656 [==============================] - 0s - loss: 0.7137 - acc: 0.6220 - val_loss: 0.0126 - val_acc: 0.9864
Epoch 1824/2000
2656/2656 [==============================] - 0s - loss: 0.0295 - acc: 0.9816 - val_loss: 0.0039 - val_acc: 0.9940
Epoch 1825/2000
2656/2656 [==============================] - 0s - loss: 0.0070 - acc: 0.9936 - val_loss: 0.0064 - val_acc: 0.9955
Epoch 1826/2000
2656/2656 [==============================] - 0s - loss: 0.0724 - acc: 0.9330 - val_loss: 0.4381 - val_acc: 0.6566
Epoch 1827/2000
2656/2656 [==============================] - 0s - loss: 0.1047 - acc: 0.9066 - val_loss: 0.0898 - val_acc: 0.9849
Epoch 1828/2000
2656/2656 [==============================] - 0s - loss: 8.8727 - acc: 0.3166 - val_loss: 

2656/2656 [==============================] - 0s - loss: 12.7200 - acc: 0.1668 - val_loss: 109.7418 - val_acc: 0.0994
Epoch 1884/2000
2656/2656 [==============================] - 0s - loss: 71.0391 - acc: 0.2180 - val_loss: 1.2170 - val_acc: 0.1958
Epoch 1885/2000
2656/2656 [==============================] - 0s - loss: 0.2452 - acc: 0.8102 - val_loss: 1.9659 - val_acc: 0.0301
Epoch 1886/2000
2656/2656 [==============================] - 0s - loss: 0.1972 - acc: 0.8727 - val_loss: 0.0048 - val_acc: 0.9940
Epoch 1887/2000
2656/2656 [==============================] - 0s - loss: 0.0065 - acc: 0.9921 - val_loss: 0.0043 - val_acc: 0.9940
Epoch 1888/2000
2656/2656 [==============================] - 0s - loss: 0.0065 - acc: 0.9925 - val_loss: 0.0038 - val_acc: 0.9940
Epoch 1889/2000
2656/2656 [==============================] - 0s - loss: 0.0131 - acc: 0.9891 - val_loss: 0.0066 - val_acc: 0.9925
Epoch 1890/2000
2656/2656 [==============================] - 0s - loss: 0.0307 - acc: 0.9699 - val_los

2656/2656 [==============================] - 0s - loss: 0.3993 - acc: 0.7413 - val_loss: 0.0098 - val_acc: 0.9955
Epoch 1947/2000
2656/2656 [==============================] - 0s - loss: 0.5605 - acc: 0.7078 - val_loss: 0.0227 - val_acc: 0.9819
Epoch 1948/2000
2656/2656 [==============================] - 0s - loss: 0.0511 - acc: 0.9559 - val_loss: 0.0276 - val_acc: 0.9925
Epoch 1949/2000
2656/2656 [==============================] - 0s - loss: 5.0895 - acc: 0.2319 - val_loss: 1.3309 - val_acc: 0.0678
Epoch 1950/2000
2656/2656 [==============================] - 0s - loss: 4.0675 - acc: 0.5840 - val_loss: 0.1394 - val_acc: 0.9172
Epoch 1951/2000
2656/2656 [==============================] - 0s - loss: 1.6517 - acc: 0.5060 - val_loss: 0.0986 - val_acc: 0.9503
Epoch 1952/2000
2656/2656 [==============================] - 0s - loss: 27.1547 - acc: 0.2684 - val_loss: 3.1008 - val_acc: 0.1130
Epoch 1953/2000
2656/2656 [==============================] - 0s - loss: 5.5950 - acc: 0.3596 - val_loss: 

In [78]:
scores = model.evaluate(X_train.values, Y_train.values) 
scores

3168/3320 [===========================>..] - ETA: 0s

[2.0121761008917569, 0.17259036144578313]

In [79]:
filename_test = "D:/Program/dataset/Taxi Trip Time Prediction/test.csv"

df_t = pd.read_csv(filename_test)
df_t.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"


In [80]:
df_t.shape

(320, 9)

In [81]:
df_t = df_t.drop(["CALL_TYPE","ORIGIN_CALL","ORIGIN_STAND","TRIP_ID","DAY_TYPE","MISSING_DATA"],axis=1)
df_t['TAXI_ID'] -= np.min(df_t['TAXI_ID']) 
df_t["timecost"] = df_t.apply(caltime,axis=1)
df_t["snapshots"] = df_t.apply(snapshot,axis=1)*3


df_t['initial_lon'] = df_t.apply(initial_long,axis=1)
df_t['initial_lat'] = df_t.apply(initial_lat,axis=1)
df_t['diff'] = df_t.apply(haversine,axis=1)

dss = df_t.apply(get_date,axis=1)
dss.columns = ["mon","day","hour"]
df_t = df_t.join(dss)

cols = df_t.columns.tolist()
y_label = cols[3]
del cols[3]
cols.append(y_label)
df_t = df_t[cols]

df_t = df_t.drop(["TIMESTAMP"],axis=1)
df_t = df_t.drop(["POLYLINE"],axis=1)
df_t = df_t.drop(["TAXI_ID"],axis=1)

X_test,Y_train = df_t[df.columns[:-1]],df_t[df.columns[-1]]
X_test = X_test.drop(["mon","day"],axis=1)

print(df_t.head())
X_test.head()

   snapshots  initial_lon  initial_lat      diff  mon   day  hour  timecost
0         33    -8.585676    41.148522  0.221328  8.0  14.0  17.0       150
1        120    -8.610876    41.145570  2.140926  8.0  14.0  17.0       585
2        120    -8.585739    41.148558  2.315694  8.0  14.0  17.0       585
3         24    -8.613963    41.141169  0.060305  8.0  14.0  17.0       105
4          6    -8.619903    41.148036  0.000754  8.0  14.0  17.0        15


,snapshots,initial_lon,initial_lat,diff,hour
0,33,-8.585676,41.148522,0.221328,17.0
1,120,-8.610876,41.145570,2.140926,17.0
2,120,-8.585739,41.148558,2.315694,17.0
3,24,-8.613963,41.141169,0.060305,17.0
4,6,-8.619903,41.148036,0.000754,17.0


In [82]:
prediction = model.predict(X_test.values)
prediction = prediction.round()

In [83]:
sub = "D:/Program/dataset/Taxi Trip Time Prediction/test.csv"
submission = pd.read_csv(sub)
submission = submission.drop(submission.columns[1:],axis=1)
submission["TRAVEL_TIME"] = prediction
submission.to_csv('submission.csv', index=False)